In [1]:
import pandas as pd
import numpy as np

In [2]:
ten_day_df = pd.read_csv("callups_10day_tidy.csv")
two_way_df = pd.read_csv("two_way_contracts.csv")
conversions_df = pd.read_csv("two_way_conversions.csv")
sportsradar_df_2024 = pd.read_csv("../raw/gleague_player_season_stats_2024_REG.csv")
sportsradar_df_2023 = pd.read_csv("../raw/gleague_player_season_stats_2023_REG.csv")
sportsradar_df_2022 = pd.read_csv("../raw/gleague_player_season_stats_2022_REG.csv")
sportsradar_df_2021 = pd.read_csv("../raw/gleague_player_season_stats_2021_REG.csv")
sportsradar_df_2019 = pd.read_csv("../raw/gleague_player_season_stats_2019_REG.csv")

ten_day_df.head(), two_way_df.head(), conversions_df.head()

(         date   season         player_name               nba_team  \
 0  2007-01-06  2006-07     Thompson, Dijon          Atlanta Hawks   
 1  2007-01-27  2006-07  Richardson, Jeremy          Atlanta Hawks   
 2  2007-04-04  2006-07     Pinkney, Kevinn         Boston Celtics   
 3  2007-04-02  2006-07       Willis, Kevin       Dallas Mavericks   
 4  2007-01-17  2006-07      Major, Renaldo  Golden State Warriors   
 
                                        contract_type  
 0      Two 10-day contracts (only 1st date is shown)  
 1      Two 10-day contracts (only 1st date is shown)  
 2                                    10-day contract  
 3  10-day contract followed by signing for the re...  
 4                                    10-day contract  ,
       player_name        date        nba_team   season    contract_type
 0      Eli Ndiaye  2025-07-01   atlanta hawks  2025-26  two-way signing
 1    Jacob Toppin  2025-07-01   atlanta hawks  2025-26  two-way signing
 2   Caleb Houston  20

In [3]:
def clean_name(name):
    if isinstance(name, str) and "," in name:
        last, first = name.split(",", 1)
        return first.strip() + " " + last.strip()
    return name.strip() if isinstance(name, str) else name

# Clean name fields
ten_day_df["player_name"] = ten_day_df["player_name"].apply(clean_name)
two_way_df["player_name"] = two_way_df["player_name"].apply(clean_name)
conversions_df["player_name"] = conversions_df["player_name"].apply(clean_name)

# SportsRadar originally uses "full_name"
sportsradar_df_2024 = sportsradar_df_2024.rename(columns={"full_name": "player_name"})
sportsradar_df_2024["player_name"] = sportsradar_df_2024["player_name"].apply(clean_name)
sportsradar_df_2023 = sportsradar_df_2023.rename(columns={"full_name": "player_name"})
sportsradar_df_2023["player_name"] = sportsradar_df_2023["player_name"].apply(clean_name)
sportsradar_df_2022 = sportsradar_df_2022.rename(columns={"full_name": "player_name"})
sportsradar_df_2022["player_name"] = sportsradar_df_2022["player_name"].apply(clean_name)
sportsradar_df_2021 = sportsradar_df_2021.rename(columns={"full_name": "player_name"})
sportsradar_df_2021["player_name"] = sportsradar_df_2021["player_name"].apply(clean_name)
sportsradar_df_2019 = sportsradar_df_2019.rename(columns={"full_name": "player_name"})
sportsradar_df_2019["player_name"] = sportsradar_df_2019["player_name"].apply(clean_name)

In [4]:
contracts_df = pd.concat([ten_day_df, two_way_df, conversions_df], ignore_index=True)
contracts_df.head()


,date,season,player_name,nba_team,contract_type
0,2007-01-06,2006-07,Dijon Thompson,Atlanta Hawks,Two 10-day contracts (only 1st date is shown)
1,2007-01-27,2006-07,Jeremy Richardson,Atlanta Hawks,Two 10-day contracts (only 1st date is shown)
2,2007-04-04,2006-07,Kevinn Pinkney,Boston Celtics,10-day contract
3,2007-04-02,2006-07,Kevin Willis,Dallas Mavericks,10-day contract followed by signing for the re...
4,2007-01-17,2006-07,Renaldo Major,Golden State Warriors,10-day contract


In [5]:
contracts_df["year"] = pd.to_datetime(contracts_df["date"]).dt.year
contracts_df["callup"] = True
contracts_df["contract_type"] = contracts_df["contract_type"].astype(str)
callups_df = contracts_df[["player_name", "year", "callup", "contract_type"]]

for df in [sportsradar_df_2024, sportsradar_df_2023, sportsradar_df_2022, sportsradar_df_2021, sportsradar_df_2019]:
    df["year"] = df["season_id"].astype(str).str[:4].astype(int)

all_stats = pd.concat([
        sportsradar_df_2024,
        sportsradar_df_2023,
        sportsradar_df_2022,
        sportsradar_df_2021,
        sportsradar_df_2019],
    ignore_index=True
)

In [6]:
final_df = all_stats.merge(
    callups_df,
    on=["player_name", "year"],
    how="left"
)
final_df["callup"] = final_df["callup"].astype("boolean").fillna(False)
final_df["contract_type"] = final_df["contract_type"].fillna("None")

final_df = final_df.sort_values(
    by=["year", "player_name"],
    ascending=[False, True]
)


In [7]:
final_df.to_csv("combined_gleague_contracts.csv", index=False)
final_df

,season_id,season_type,team_id,player_id,player_name,position,total_games_played,total_games_started,total_minutes,total_field_goals_made,...,turnovers,fgm,fga,tpm,tpa,ftm,fta,year,callup,contract_type
2,2024,REG,2d0a4879-33d3-4af9-a12f-8081362ff548,175b1d50-ef8d-4d65-a46a-864d73342aab,A.J. Lawson,G,23,21,740.0,152,...,46,152,351,55,145,46,71,2024,False,None
430,2024,REG,1adf6996-af2e-41a2-b158-174b0e4c5a1d,2cd254c6-37a4-4ac6-a343-093b1eaf282f,A.J. Reeves,G,3,0,37.0,6,...,1,6,10,4,8,1,2,2024,False,None
503,2024,REG,a565062b-f8be-4a7f-b1f1-502057f3b365,2cd254c6-37a4-4ac6-a343-093b1eaf282f,A.J. Reeves,G,17,2,223.0,34,...,6,34,87,25,67,7,8,2024,False,None
217,2024,REG,cefdc308-b8ea-441e-a698-6ef3891cec35,3a258e45-3005-429e-a544-6bbc097c0640,AJ Johnson,G,5,5,149.0,29,...,10,29,64,13,31,8,9,2024,False,None
250,2024,REG,9ea5b326-cd0b-4bc2-bcec-dc174cb22ae9,3a258e45-3005-429e-a544-6bbc097c0640,AJ Johnson,G,15,13,422.0,73,...,43,73,168,26,61,14,20,2024,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517,2019,REG,4e26f748-e98f-412e-9c27-f04b255e7f25,873a1ec0-28a0-489e-b4a0-8152eee164aa,Yuta Watanabe,G-F,22,22,719.0,149,...,26,149,277,32,88,31,38,2019,False,None
2406,2019,REG,0a3d428b-572a-4a94-b716-8c1297c53d8e,ba506d40-afcb-4fca-8323-6a2485ea9a87,Zach Norvell Jr.,G-F,29,17,861.0,153,...,63,153,377,86,217,32,39,2019,False,None
2698,2019,REG,b0a52b1f-878d-4987-9249-026360e0207a,ba506d40-afcb-4fca-8323-6a2485ea9a87,Zach Norvell Jr.,G-F,7,6,212.0,30,...,12,30,84,22,61,3,4,2019,False,None
2505,2019,REG,4dc54726-0738-413b-98b1-5b90dfe99aae,bb3bbd62-472b-42b8-b9d4-f2dc661f631b,Zak Irvin,G,41,23,1103.0,179,...,72,179,313,2,26,25,43,2019,False,None
